# Imports

In [60]:
import numpy as np
import matplotlib.pyplot as plt
import numba
from numba import njit
from scipy.ndimage import convolve

In [82]:
def get_energy(lattice, N, J1, J2):
    kernel_nn = np.array([
                [0, 1, 0],
                [1, 0, 1],
                [0, 1, 0]])
    
    kernel_nnn = np.array([
                [1, 0, 1],
                [0, 0, 0],
                [1, 0, 1]])
    
    energy_nn = -J1 * lattice * convolve(lattice, kernel_nn, mode='wrap')
    energy_nnn = -J2 * lattice * convolve(lattice, kernel_nnn, mode='wrap')
    
    return (energy_nn + energy_nnn).sum()/(2*N)

In [88]:
@njit
def get_dE(lattice, x, y, N, J1, J2):
    nn_sum = (
        lattice[(x-1)%N, y] + lattice[(x+1)%N, y] +
        lattice[x, (y-1)%N] + lattice[x, (y+1)%N]
    )

    nnn_sum = (
        lattice[(x-1)%N, (y-1)%N] + lattice[(x+1)%N, (y-1)%N] +
        lattice[(x-1)%N, (y+1)%N] + lattice[(x+1)%N, (y+1)%N]
    )

    dE = 2 * lattice[x, y] * (J1 * nn_sum + J2 * nnn_sum)

    return dE

In [99]:
# @numba.njit("UniTuple(f8[:], 2)(f8[:,:], i8, f8, f8, i8, f8, f8)", nogil=True)
@njit
def metropolis(lattice, MC_steps, T, energy, N, J1, J2, save_images=False, image_spacing=None):
    web = lattice.copy()
    net_spins = np.zeros(MC_steps-1)
    net_energy = np.zeros(MC_steps-1)

    if save_images and image_spacing is not None:
        images = np.empty((len(image_spacing), N, N), dtype=np.int8)
        aux_img_idx = 0

    else:
        images = None
        image_indices = None


    for t in range(0,MC_steps-1):

        if save_images and t in image_spacing:
            images[aux_img_idx] = web.copy()
            aux_img_idx += 1

        # 2. pick random point on array and flip spin
        x = np.random.randint(0,N)
        y = np.random.randint(0,N)

        
        dE = get_dE(web, x, y, N, J1, J2)
        if (dE>0)*(np.random.random() < np.exp(-T*dE)):
            web[x,y] *= -1
            energy += dE
        elif dE<=0:
            web[x,y] *= -1
            energy += dE
            
        net_spins[t] = web.sum()/(N**2)
        net_energy[t] = energy
            
    return net_spins, net_energy, images, image_indices

In [129]:
SEED = 42
np.random.seed(SEED)

N = 50
J1 = 0.5
J2 = 1.0
T = 0.5
MC_steps = 1000000

In [130]:
# Generate a random 2D lattice

# Probability of value 1 (-1 has (p-1) probability)
p_1 = 0.5
lattice_1 = np.random.choice([1, -1], size=(N, N), p=[p_1, 1-p_1])

# Generate another lattice with a different probability
p_2 = 0.2
lattice_2 = np.random.choice([1, -1], size=(N, N), p=[p_2, 1-p_2])

In [131]:
energy_0 = get_energy(lattice_1, N, J1, J2)
num_imgs = MC_steps // 1000
img_spacing = np.unique(np.logspace(0, np.log10(MC_steps), num=num_imgs, dtype=int))
spins, energies, images, image_indices = metropolis(lattice_1, MC_steps, T, energy_0, N, J1, J2, save_images=True, image_spacing=img_spacing)

In [132]:
import imageio

# Create a GIF from the images array
gif_filename = "simulation.gif"
with imageio.get_writer(gif_filename, mode="I", duration=0.1) as writer:
    for img in images:
        writer.append_data(((img + 1) * 127.5).astype(np.uint8).repeat(10, axis=0).repeat(10, axis=1))  # Scale values and enlarge

print(f"GIF saved as {gif_filename}")

GIF saved as simulation.gif
